# Ejemplo de un Sistema de Recomendación

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oramosul/abd-files/blob/main/spark/3-spark-mllib/6-Sistema-recomendacion.ipynb)

Se usará datos adaptados de movielens (https://grouplens.org/datasets/movielens/). Dados nuevos usuarios, se desea predecir su "rating" sobre películas que aún no han visto (IDpelicula), con base en usuarios anteriores.


In [1]:
# Solo necesario si se usa Google Colab
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [3]:
# Carga de archivos
!wget -q https://raw.githubusercontent.com/oramosul/abd-files/main/spark/datos/ratings_peliculas.csv

In [4]:
# Lectura de datos
df = spark.read.csv('ratings_peliculas.csv', inferSchema=True, header=True)
df.show(5)

+----------+------+---------+
|IDpelicula|rating|IDusuario|
+----------+------+---------+
|         2|   3.0|        0|
|         3|   1.0|        0|
|         5|   2.0|        0|
|         9|   4.0|        0|
|        11|   1.0|        0|
+----------+------+---------+
only showing top 5 rows



In [5]:
# Dividir los datos en entrenamiento y prueba (80% - 20%)
df_train, df_test = df.randomSplit([0.8, 0.2])
df_train.show(5)

+----------+------+---------+
|IDpelicula|rating|IDusuario|
+----------+------+---------+
|         0|   1.0|        3|
|         0|   1.0|        5|
|         0|   1.0|        6|
|         0|   1.0|        8|
|         0|   1.0|       11|
+----------+------+---------+
only showing top 5 rows



In [6]:
# Observar la cantidad de usuarios y de películas
df_train.describe().show()

+-------+------------------+------------------+------------------+
|summary|        IDpelicula|            rating|         IDusuario|
+-------+------------------+------------------+------------------+
|  count|              1219|              1219|              1219|
|   mean| 49.43478260869565|1.7768662838392124|14.663658736669401|
| stddev|29.188048440607645|1.1874091890622678| 8.558506272942845|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



### Modelo ALS: Alternating Least Squares matrix factorization

In [7]:
# Construir el modelo usando ALS
from pyspark.ml.recommendation import ALS

als = ALS(userCol="IDusuario", itemCol="IDpelicula",
          ratingCol="rating",
          maxIter=5, regParam=0.01)

modelo = als.fit(df_train)

### Predicciones

In [8]:
# Predicciones sobre el conjunto de prueba
predicciones = modelo.transform(df_test)

predicciones.show(10)

+----------+------+---------+----------+
|IDpelicula|rating|IDusuario|prediction|
+----------+------+---------+----------+
|         0|   3.0|       28|0.83757824|
|         0|   1.0|       22| 0.4381315|
|         3|   1.0|       13| 1.6260742|
|         1|   1.0|        6| 1.6903968|
|         2|   1.0|       19| 2.0985105|
|         1|   1.0|        4| 1.8222469|
|         0|   1.0|       23|0.74805903|
|         2|   4.0|       10| 1.2512517|
|         0|   1.0|       21|-1.0585362|
|         2|   4.0|       21| 1.2664464|
+----------+------+---------+----------+
only showing top 10 rows



In [9]:
# Predicción para un solo usuario (usuario 11): se mantiene solo IDusuario y IDpellicula
usuario = df_test.filter(df_test['IDusuario']==11).select(['IDpelicula','IDusuario'])

# Muestra el usuario y las películas que podría ver
usuario.show()

# Recomendaciones de las películas que vería
recomendaciones = modelo.transform(usuario)

# Mostrar resultados ordenados
recomendaciones.orderBy('prediction', ascending=False).show()

+----------+---------+
|IDpelicula|IDusuario|
+----------+---------+
|         6|       11|
|        20|       11|
|        32|       11|
|        45|       11|
|        50|       11|
|        64|       11|
|        72|       11|
|        81|       11|
+----------+---------+

+----------+---------+------------+
|IDpelicula|IDusuario|  prediction|
+----------+---------+------------+
|        32|       11|   2.2664478|
|        64|       11|   2.0174384|
|        81|       11|   1.4366398|
|        50|       11|   1.3724024|
|         6|       11|   0.7235401|
|        20|       11|   0.7183197|
|        45|       11|-0.027657092|
|        72|       11|  -1.8631816|
+----------+---------+------------+



### Evaluación

Se brinda una medida referencial. Sin embargo, es bastante complicado saber de manera concluyente cuán bien un sistema de recomendación funciona, dado que hay mucha subjetividad del usuario.

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator

# Se usará RMSE (Root Mean Squared Error) para evaluar la predicción vs rating
evaluador = RegressionEvaluator(labelCol="rating", predictionCol="prediction",
                                metricName="rmse")

rmse = evaluador.evaluate(predicciones)
print("Error RMSE: {:.2f}".format(rmse))

Error RMSE: 1.75
